### Film и crop в массив

на основе scikit-video

In [ ]:
!pip install scikit-video -q
!pip install einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00


# common_metrics_on_video_quality
https://github.com/JunyaoHu/common_metrics_on_video_quality/tree/main

In [ ]:
!pip install lpips -q
!git clone https://github.com/JunyaoHu/common_metrics_on_video_quality.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 662.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.5 MB/s eta 0:00:00


In [ ]:
import os
import torch
import sys
dir_project = '/content/common_metrics_on_video_quality'
sys.path.append(dir_project)

import skvideo.io
import numpy as np
np.float = np.float64
np.int = np.int_
import json
from calculate_fvd import calculate_fvd
from calculate_psnr import calculate_psnr
from calculate_ssim import calculate_ssim
from calculate_lpips import calculate_lpips
device = torch.device("cuda")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 89.7MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:
MAIN_DIR = " you main dir"
EVAL_DIR = MAIN_DIR + "/Text2Video_Zero"

MOVI_DIR = MAIN_DIR + "/MSR_VIT_movi"
ORIG_DIR = MOVI_DIR + "/crop_hand_selected_mp4"

SIZE = 256
QTYFRAMES = 16

def movi_numpy_to_tensor(array):
    return torch.tensor(array).permute(0, 3, 1, 2).unsqueeze(0)

## real_video

In [ ]:
video_list = os.listdir(ORIG_DIR)
video_list.sort()
print(len(video_list))

video_directory = ORIG_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

real_video = torch.concat(all_video)
real_video.shape

31


torch.Size([31, 16, 3, 256, 256])

#  Fréchet Video Distance (FVD)

## predict_chunks_H512_W512_F16_caption_uniform

In [ ]:
GEN_DIR = EVAL_DIR + "/predict_chunks_H512_W512_F16_caption_uniform"
video_list = os.listdir(GEN_DIR)
video_list.sort()
print(len(video_list))

video_directory = GEN_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

predict_chunks = torch.concat(all_video)
predict_chunks.shape

31


torch.Size([31, 16, 3, 256, 256])

In [ ]:
result = {}
result['fvd_VGPT'] = calculate_fvd(real_video/255, predict_chunks/255, device, method='videogpt')
result['fvd_GAN'] = calculate_fvd(real_video/255, predict_chunks/255, device, method='styleganv')
result['ssim'] = calculate_ssim(real_video/255, predict_chunks/255)
result['psnr'] = calculate_psnr(real_video/255, predict_chunks/255)
result['lpips'] = calculate_lpips(real_video/255, predict_chunks/255, device)
print(json.dumps(result, indent=4))

calculate_fvd...
/content/common_metrics_on_video_quality/fvd/videogpt/i3d_pretrained_400.pt


100%|██████████| 7/7 [00:34<00:00,  4.91s/it]


calculate_fvd...
/content/common_metrics_on_video_quality/fvd/styleganv/i3d_torchscript.pt


100%|██████████| 7/7 [00:29<00:00,  4.16s/it]


calculate_ssim...


...

calculate_psnr...


100%|██████████| 31/31 [00:00<00:00, 105.35it/s]


calculate_lpips...


100%|██████████| 31/31 [00:02<00:00, 15.19it/s]


{
    "fvd_VGPT": {
        "value": {
            "10": 2174.69189453125,
            "11": 2209.57568359375,
            "12": 2151.262451171875,
            "13": 2103.23486328125,
            "14": 2129.16796875,
            "15": 1902.8331298828125,
            "16": 2055.94775390625
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"
    },
    "fvd_GAN": {
        "value": {
            "10": -60129541813.37057,
            "11": -5803.118055507167,
            "12": 218104155.4603069,
            "13": 8388961.126125915,
            "14": -9699010.597300952,
            "15": 16661.1649788001,
            "16": 6.293109725842669
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"
   

In [ ]:
import numpy as np

In [ ]:
d = {            "10": 2174.69189453125,
            "11": 2209.57568359375,
            "12": 2151.262451171875,
            "13": 2103.23486328125,
            "14": 2129.16796875,
            "15": 1902.8331298828125,
            "16": 2055.94775390625
                 }

res = np.array(list(d.values()))
print(f"FVD predict_step_video mean: {np.mean(res )}")
print(f"FVD predict_step_video std: {np.std(res )}")

FVD predict_step_video mean: 2103.816249302455
FVD predict_step_video std: 93.9515163200421


In [ ]:
d = {
            "0": 0.20277474091113293,
            "1": 0.22353910835413157,
            "2": 0.19996373837376177,
            "3": 0.20810065715710724,
            "4": 0.21184800992905828,
            "5": 0.19791071521059267,
            "6": 0.22666583539059432,
            "7": 0.23082670027465133,
            "8": 0.24204541214913156,
            "9": 0.21678232815345896,
            "10": 0.23099837993375918,
            "11": 0.21958738939320116,
            "12": 0.2149399468187004,
            "13": 0.20452230733172608,
            "14": 0.21531695384078825,
            "15": 0.20038782245304948
                 }

res = np.array(list(d.values()))
print(f"SSIM predict_step_video mean: {np.mean(res )}")
print(f"SSIM  predict_step_video std: {np.std(res )}")

SSIM predict_step_video mean: 0.2153881278546778
SSIM  predict_step_video std: 0.01254300887008023


In [ ]:
d = {
            "0": 7.574471956704536,
            "1": 7.4914086609226125,
            "2": 7.120068326286014,
            "3": 7.015331353616517,
            "4": 7.033799187417488,
            "5": 7.043367326084667,
            "6": 7.236158818422363,
            "7": 7.4015660901649545,
            "8": 7.112860253384629,
            "9": 7.249649871598288,
            "10": 6.988234170323534,
            "11": 6.976453344410852,
            "12": 6.72156229179362,
            "13": 6.868925075444366,
            "14": 6.951608401215786,
            "15": 6.503357074870868
                 }

res = np.array(list(d.values()))
print(f"PSNR predict_step_video mean: {np.mean(res )}")
print(f"PSNR  predict_step_video std: {np.std(res )}")

PSNR predict_step_video mean: 7.080551387666318
PSNR  predict_step_video std: 0.26479289901904296


In [ ]:
d = {
            "0": 0.8296420228096747,
            "1": 0.8350869244144808,
            "2": 0.8432596768102338,
            "3": 0.8444397141856532,
            "4": 0.8585627963466029,
            "5": 0.8618254353923183,
            "6": 0.8563660498588316,
            "7": 0.8551902559495741,
            "8": 0.8647849886648117,
            "9": 0.8647298909002735,
            "10": 0.8597648797496673,
            "11": 0.8556873586870009,
            "12": 0.8533101197211973,
            "13": 0.8607741786587623,
            "14": 0.8530583670062404,
            "15": 0.8629079018869708
                 }

res = np.array(list(d.values()))
print(f"LPIPS predict_step_video mean: {np.mean(res )}")
print(f"LPIPS  predict_step_video std: {np.std(res )}")

LPIPS predict_step_video mean: 0.8537119100651434
LPIPS  predict_step_video std: 0.010123126316980821


# Frechet Inception Distance (FID)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q torchmetrics[image]
from torchmetrics.image.fid import FrechetInceptionDistance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 19.7 MB/s eta 0:00:00


## predict_chunks_H512_W512_F16_caption_uniform

In [ ]:
fid_eval = []
for i in range(real_video.shape[0]):
    fid = FrechetInceptionDistance(normalize=True)
    real_images = real_video[i]
    fake_images = predict_chunks[i]
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)
    eval = float(fid.compute())
    fid_eval.append(eval)
    print(f"FID: {eval}")

fid_eval = np.array(fid_eval)
print(f"FID predict_step_video mean: {np.mean(fid_eval)}")
print(f"FID predict_step_video std: {np.std(fid_eval)}")

FID: 410.4081115722656
FID: 393.0689392089844
FID: 484.0835876464844
FID: 385.1629638671875
FID: 402.00189208984375
FID: 272.086669921875
FID: 314.02099609375
FID: 417.7732238769531
FID: 390.4472351074219
FID: 618.8546752929688
FID: 459.6355895996094
FID: 408.7217712402344
FID: 318.37445068359375
FID: 473.2254943847656
FID: 478.6835021972656
FID: 436.06463623046875
FID: 411.4712829589844
FID: 489.8173522949219
FID: 305.798828125
FID: 463.31427001953125
FID: 396.2080078125
FID: 460.93603515625
FID: 441.8587951660156
FID: 388.5010681152344
FID: 394.36346435546875
FID: 550.5819091796875
FID: 400.22198486328125
FID: 426.98797607421875
FID: 495.7095031738281
FID: 431.3387451171875
FID: 436.1044921875
FID predict_step_video mean: <built-in method mean of numpy.ndarray object at 0x7bf813022cd0>
FID predict_step_video std: 68.58107805174338


In [ ]:
print(f"FID predict_step_video mean: {np.mean(fid_eval)}")
print(f"FID predict_step_video std: {np.std(fid_eval)}")

FID predict_step_video mean: 424.38153076171875
FID predict_step_video std: 68.58107805174338
